## Stack OViN

To avoid Covid data, years before 2020 will be used.

Start with 2013-2017 (OViN), can maybe add 2018-2019 (ODiN) later??

For OViN -> 2017 base year

In [ ]:
import pandas as pd
import numpy as np
import os

%matplotlib inline

Load the data

In [ ]:
parent_dir = os.path.split(os.getcwd())[0]

In [ ]:
ovin13 = pd.read_csv(parent_dir + '\Data\ODiN_OViN\OViN2013_Databestand.tab', delimiter='\t')
ovin14 = pd.read_csv(parent_dir + '\Data\ODiN_OViN\OViN2014_Databestand.tab', delimiter='\t')
ovin15 = pd.read_csv(parent_dir + '\Data\ODiN_OViN\OViN2015_Databestand.tab', delimiter='\t')
ovin16 = pd.read_csv(parent_dir + '\Data\ODiN_OViN\OViN2016_Databestand.tab', delimiter='\t')
ovin17 = pd.read_csv(parent_dir + '\Data\ODiN_OViN\OViN2017_Databestand.tab', delimiter='\t')


Total population for each year. (Source cbs.statline)

In [ ]:
tot_pop = {2013:16804432, 2014:16865008, 2015:16939923, 2016:17030314, 
           2017:17131296, 2018:17231624, 2019:17344874}

Determine weightfactors for each year (2017 is base year)

In [ ]:
weight_pop = {}

for key in tot_pop:
    weight_pop[key] = tot_pop[2017] / tot_pop[key]

Filter workdays

In [ ]:
ovin = {2013: ovin13,
        2014: ovin14,
        2015: ovin15,
        2016: ovin16,
        2017: ovin17}

In [ ]:
ovin[2017]= ovin[2017].rename(columns={'op':'OP'}) # Rename 'OP' column

In [ ]:
tot_ovin_pop = {}

for year in ovin:
    
    tot_ovin_pop[year] = ovin[year][ovin[year].OP == 1].FactorP.sum()

tot_ovin_pop

In [ ]:
ovin[2013]

Select and remove holidays

In [ ]:
holiday = [[1, 1], [5, 5], [25, 12], [26, 12]]
holiday_dict = {2013:[[30, 4], [1, 4], [9, 5], [10, 5], [20, 5]],
                2014:[[27, 4], [21, 4], [29, 5], [30, 5], [9 ,6]],
                2015:[[27, 4], [6, 4], [14 ,5], [15, 5], [25, 5]],
                2016:[[27, 4], [28, 3], [6, 5], [16, 5]],
                2017:[[27, 4], [17, 4], [25, 5], [26, 5], [5, 6]]}

In [ ]:
# Loop over all years
for year in ovin:
    holiday_dict[year] = holiday + holiday_dict[year]

    # Loop over all holidays
    for day in holiday_dict[year]:
        
        # If day is already removed, ignore checks
        if len(ovin[year][(ovin[year].Dag == day[0]) & (ovin[year].Maand == day[1])]) == 0:
            pass
        
        # If holiday is on a Tuesday, also remove the Monday
        elif (ovin[year][(ovin[year].Dag == day[0]) & (ovin[year].Maand == day[1])].iloc[0].Weekdag == 3):
            print(f'{day} is on Tuesday in {year}')
            if day[0] > 1:  
                ovin[year] = ovin[year][(ovin[year].Dag != day[0] - 1) | (ovin[year].Maand != day[1])]
                
    	# If holiday is on a Thurday, also remove the Thurday
        elif (ovin[year][(ovin[year].Dag == day[0]) & (ovin[year].Maand == day[1])].iloc[0].Weekdag == 5):
            print(f'{day} is on Thursday in {year}')
            holiday_dict[year].append([day[0] + 1, day[1]])
            ovin[year] = ovin[year][(ovin[year].Dag != day[0] + 1) | (ovin[year].Maand != day[1])]

        # Remove the holiday
        ovin[year] = ovin[year][(ovin[year].Dag != day[0]) | (ovin[year].Maand != day[1])]
    
    ovin[year] = ovin[year][ovin[year]['Weekdag'].isin(np.arange(2, 7))] # Keep only the workdays for each year

    

### Create new weight factors

1. Reweigh FactorV to workday

First, get the number of workdays

In [ ]:
workday_dict = {}

for year in ovin:
    workdays = 0
    
    for month in range(1, 13):
        workdays += len(set(ovin[year][ovin[year].Maand == month].Dag))
    
    workday_dict[year] = workdays

workday_dict

And then calculate Factor for each workday

In [ ]:
for year in ovin:

    ovin[year]['FactorV_wd'] = ovin[year]['FactorV'] / workday_dict[year]

2. Scale FactorV_wd to base year

In [ ]:
for year in ovin:

    ovin[year]['FactorV_wd_bj'] = ovin[year]['FactorV_wd'] * weight_pop[year]


3. Make data representative for whole OViN population

In [ ]:
# Determine tot persons in OViN year:
ovin_pop = {}

for year in ovin:
    ovin_pop[year] = ovin[year][ovin[year].OP == 1].FactorV_wd_bj.sum()

ovin_pop

Keep only verplaatsingen data points that indicate a new verplaatsing (so people who make no verplaatsingen are omited)

In [ ]:
for year in ovin:
    ovin[year] = ovin[year][~ovin[year].Verpl.isnull()]
    ovin[year] = ovin[year][ovin[year].Verpl != 0]
# ovin[2013][~ovin[2013].Verpl.isnull()]

In [ ]:
ovin[2013][ovin[2013].Verpl == 0]

In [ ]:
# Determine tot verplaatsingen
ovin_verpl = {}

for year in ovin:
    ovin_verpl[year] = ovin[year].FactorV_wd_bj.sum()

ovin_verpl


In [ ]:
# Determine no. of verpl per person
verplPP = {}

for year in ovin:
    verplPP[year] = ovin_verpl[year] / ovin_pop[year]


verplPP

In [ ]:
# Determine tot verpl
totAB_Verpl = {}

for year in ovin:
    totAB_Verpl[year] = verplPP[year] * tot_ovin_pop[year]


totAB_Verpl

In [ ]:
ovin[2013].FactorV_wd_bj.sum()

In [ ]:
tot_verpl = np.sum(list(totAB_Verpl.values()))

share_verpl = {}

for year in ovin:
    share_verpl[year] = totAB_Verpl[year] / tot_verpl


share_verpl

Divide all factors thourgh 5, to get the same total number of verplaatsingen whens stacking the dataset

In [ ]:
for year in ovin:
    ovin[year]['FactorV_wd_stack'] = ovin[year]['FactorV_wd_bj'] / 5

In [ ]:
ovin_tot = pd.concat([ovin[2013], ovin[2014], ovin[2015], ovin[2016], ovin[2017]], ignore_index=True)

In [ ]:
ovin_tot.FactorV_wd_stack.sum()

In [ ]:
ovin_verpl

## Missing postcodes

In [ ]:
missing_PC = ovin_tot[(ovin_tot['AankPC'] == '0000') | (ovin_tot['VertPC'] == '0000') | 
                      (ovin_tot['AankPC'] == 0) | (ovin_tot['VertPC'] == 0)| 
                      (ovin_tot['AankPC'] == '0') | (ovin_tot['VertPC'] == '0')]
print(len(missing_PC))

scale = len(ovin_tot) / (len(ovin_tot) - (len(missing_PC)))
scale = ovin_tot.FactorV_wd_stack.sum() / (ovin_tot.FactorV_wd_stack.sum() - missing_PC.FactorV_wd_stack.sum())
print(scale)

ovin_tot['FactorV_final'] = ovin_tot['FactorV_wd_stack'] * scale
                

In [ ]:
missing_PC[['AankPC', 'VertPC']]

In [ ]:
# Delete missing values
ovin_tot = ovin_tot[~((ovin_tot['AankPC'] == '0000') | (ovin_tot['VertPC'] == '0000') | 
                      (ovin_tot['AankPC'] == 0) | (ovin_tot['VertPC'] == 0)| 
                      (ovin_tot['AankPC'] == '0') | (ovin_tot['VertPC'] == '0'))]

In [ ]:
ovin_tot

## Save data

In [ ]:
ovin_tot.to_csv(parent_dir + '\\Data\\New\\merged_ovin.csv')